In [1]:
import yaml
import numpy as np
import matplotlib.pyplot as plt
from os import listdir

# Fit folder
FitFolders = ["/Users/s2569857/Codes/NTK-interface/build/run/Results/test",]

In [272]:
# This function makes sense only with one single output and
# is not as general as the the two other functions defined below.dd
def extract_NTK (FitFolder, tr_step, rep):
    with open(FitFolder + "/meta.yaml", "r") as meta:
        Meta = yaml.load(meta, Loader=yaml.CLoader)


    with open(FitFolder + "/log/replica_" + str(rep) + ".yaml", "r") as rep:
        Rep = yaml.load(rep, Loader=yaml.CLoader)

    size_of_data = len(Rep[tr_step]['dNN'].keys())
    NTK_ab = np.ndarray((size_of_data, size_of_data), dtype=float)

    for a in range(size_of_data):
        for b in range(size_of_data):
            NTK_ab[a][b] = Rep[tr_step]["NTK"][b + a * size_of_data]

    return NTK_ab


def first_derivative (FitFolder, tr_step, rep):
    with open(FitFolder + "/meta.yaml", "r") as meta:
        Meta = yaml.load(meta, Loader=yaml.CLoader)
        nsteps = Meta["max_num_iterations"]
    
    if tr_step > nsteps:
        raise Exception(f"The chosen time-step is too high. The maximum number of steps is {nsteps}")

    with open(FitFolder + "/log/replica_" + str(rep) + ".yaml", "r") as rep:
        Rep = yaml.load(rep, Loader=yaml.CLoader)

    size_of_data = len(Rep[tr_step]['dNN'].keys())
    size_of_output = Meta['NNarchitecture'][-1]
    number_of_derivatives = len(list(Rep[tr_step]['dNN'].values())[0]) - size_of_output
    derivative_tensor = np.ndarray((size_of_data, size_of_output, number_of_derivatives), dtype=float)     

    for id in range(size_of_data):
        for io in range(size_of_output):
            for ip in range(1, number_of_derivatives+1):
                derivative_tensor[id][io][ip-1] = list(Rep[tr_step]['dNN'].values())[id][io + ip * size_of_output]

    return derivative_tensor


def second_derivative (FitFolder, tr_step, rep):
    with open(FitFolder + "/meta.yaml", "r") as meta:
        Meta = yaml.load(meta, Loader=yaml.CLoader)
        nsteps = Meta["max_num_iterations"]
    
    if tr_step > nsteps:
        raise Exception(f"The chosen time-step is too high. The maximum number of steps is {nsteps}")

    with open(FitFolder + "/log/replica_" + str(rep) + ".yaml", "r") as rep:
        Rep = yaml.load(rep, Loader=yaml.CLoader)

    size_of_data = len(Rep[tr_step]['dNN'].keys())
    size_of_output = Meta['NNarchitecture'][-1]
    number_of_derivatives = len(list(Rep[tr_step]['dNN'].values())[0]) - size_of_output
    derivative_tensor = np.ndarray((size_of_data, size_of_output, number_of_derivatives, number_of_derivatives), dtype=float)     

    for id in range(size_of_data):
        for io in range(size_of_output):
            for ip1 in range(number_of_derivatives):
                for ip2 in range(1, number_of_derivatives+1):
                    derivative_tensor[id][io][ip1][ip2-1] = list(Rep[tr_step]['ddNN'].values())[id][ip1][io + ip2 * size_of_output]

    return derivative_tensor

In [238]:
from abc import ABC, abstractmethod

class Observable(ABC):

    def __init__(self, FitFolder):
        self.FitFolder = FitFolder
        self.number_of_replicas = len(listdir(FitFolder + "/log/"))

        with open(FitFolder + "/meta.yaml", "r") as meta:
            Meta = yaml.load(meta, Loader=yaml.CLoader)

        with open(FitFolder + "/log/replica_" + str(0) + ".yaml", "r") as rep:
            Rep = yaml.load(rep, Loader=yaml.CLoader)

        self.size_of_data = len(Rep[0]['dNN'].keys())
        self.size_of_output = Meta['NNarchitecture'][-1]


    @abstractmethod
    def Evaluate(self, replica, tr_step):
        pass
        
    def average_over_replicas(self, tr_step):

        shape = self.shape
        obs = np.zeros(shape, dtype=float)

        obs = np.average([self.Evaluate(frep, tr_step) for frep in range(self.number_of_replicas)], axis = -1)
        std = np.std([self.Evaluate(frep, tr_step) for frep in range(self.number_of_replicas)], axis = -1)
        obs = np.reshape(obs, shape)
        std = np.reshape(std, shape)

        return obs, std

In [244]:
class ddf_df_df(Observable):
    def __init__ (self, FitFolder):
        super().__init__(FitFolder)
        self.shape = (self.size_of_data, self.size_of_data, self.size_of_data, self.size_of_output, self.size_of_output, self.size_of_output)

    def Evaluate(self, replica, tr_step):
        df = first_derivative(self.FitFolder, tr_step, replica)
        ddf = second_derivative(self.FitFolder, tr_step, replica)
        df_df = np.tensordot(df, df, axes = 0)
        return np.reshape(np.tensordot(df_df, ddf, axes = ([2,5],[2,3])), self.shape)
    

class NTK_frob(Observable):
    def __init__(self, FitFolder):
        super().__init__(FitFolder)
        self.shape = (1)
    
    def Evaluate (self, replica, tr_step):
        obs = np.linalg.norm(extract_NTK(self.FitFolder, tr_step, replica))
        return obs